# Bias Propagation Analysis

This notebook analyzes the results of bias injection experiments to understand:
- How biased users affect overall recommendation quality
- Bias propagation patterns in recommendations
- Threshold effects and scalability

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

OSError: 'seaborn-v0_8' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

## 1. Load Results

In [ ]:
# Load experimental results
results_df = pd.read_csv('../results/biased/bias_injection_results.csv')

print(f"📊 Loaded {len(results_df)} experimental results")
print(f"🧪 Experiments: {results_df['genre'].value_counts().to_dict()}")

# Display first few results
display(results_df.head())

## 2. Overall Performance Analysis

In [ ]:
# Plot RMSE changes with bias injection
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# RMSE by number of biased users
baseline_rmse = results_df[results_df['genre'] == 'baseline']['overall_rmse'].iloc[0]

for genre in ['adventure', 'mystery']:
    genre_data = results_df[results_df['genre'] == genre].sort_values('num_biased_users')
    
    axes[0,0].plot(genre_data['num_biased_users'], genre_data['overall_rmse'], 
                   marker='o', label=f'{genre.title()} bias', linewidth=2)

axes[0,0].axhline(y=baseline_rmse, color='red', linestyle='--', alpha=0.7, label='Baseline')
axes[0,0].set_xlabel('Number of Biased Users')
axes[0,0].set_ylabel('RMSE')
axes[0,0].set_title('Impact on Overall RMSE')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# MAE by number of biased users
baseline_mae = results_df[results_df['genre'] == 'baseline']['overall_mae'].iloc[0]

for genre in ['adventure', 'mystery']:
    genre_data = results_df[results_df['genre'] == genre].sort_values('num_biased_users')
    
    axes[0,1].plot(genre_data['num_biased_users'], genre_data['overall_mae'], 
                   marker='o', label=f'{genre.title()} bias', linewidth=2)

axes[0,1].axhline(y=baseline_mae, color='red', linestyle='--', alpha=0.7, label='Baseline')
axes[0,1].set_xlabel('Number of Biased Users')
axes[0,1].set_ylabel('MAE')
axes[0,1].set_title('Impact on Overall MAE')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Adventure recommendation percentage
baseline_adv_pct = results_df[results_df['genre'] == 'baseline']['rec_adventure_pct'].iloc[0]

for genre in ['adventure', 'mystery']:
    genre_data = results_df[results_df['genre'] == genre].sort_values('num_biased_users')
    
    axes[1,0].plot(genre_data['num_biased_users'], genre_data['rec_adventure_pct'], 
                   marker='o', label=f'{genre.title()} bias', linewidth=2)

axes[1,0].axhline(y=baseline_adv_pct, color='red', linestyle='--', alpha=0.7, label='Baseline')
axes[1,0].set_xlabel('Number of Biased Users')
axes[1,0].set_ylabel('Adventure Recommendations (%)')
axes[1,0].set_title('Adventure Bias Propagation')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Mystery recommendation percentage
baseline_mys_pct = results_df[results_df['genre'] == 'baseline']['rec_mystery_pct'].iloc[0]

for genre in ['adventure', 'mystery']:
    genre_data = results_df[results_df['genre'] == genre].sort_values('num_biased_users')
    
    axes[1,1].plot(genre_data['num_biased_users'], genre_data['rec_mystery_pct'], 
                   marker='o', label=f'{genre.title()} bias', linewidth=2)

axes[1,1].axhline(y=baseline_mys_pct, color='red', linestyle='--', alpha=0.7, label='Baseline')
axes[1,1].set_xlabel('Number of Biased Users')
axes[1,1].set_ylabel('Mystery Recommendations (%)')
axes[1,1].set_title('Mystery Bias Propagation')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Quantify Bias Propagation Effects

In [ ]:
# Calculate relative changes from baseline
baseline_row = results_df[results_df['genre'] == 'baseline'].iloc[0]

biased_results = results_df[results_df['genre'] != 'baseline'].copy()

# Calculate percentage changes
biased_results['rmse_change_pct'] = ((biased_results['overall_rmse'] - baseline_row['overall_rmse']) / baseline_row['overall_rmse']) * 100
biased_results['mae_change_pct'] = ((biased_results['overall_mae'] - baseline_row['overall_mae']) / baseline_row['overall_mae']) * 100

biased_results['adv_rec_change'] = biased_results['rec_adventure_pct'] - baseline_row['rec_adventure_pct']
biased_results['mys_rec_change'] = biased_results['rec_mystery_pct'] - baseline_row['rec_mystery_pct']

print("📊 BIAS PROPAGATION SUMMARY")
print("=" * 50)

for genre in ['adventure', 'mystery']:
    genre_data = biased_results[biased_results['genre'] == genre]
    
    print(f"\n🎭 {genre.upper()} BIAS:")
    
    # Find maximum effects
    max_users = genre_data['num_biased_users'].max()
    max_effect = genre_data[genre_data['num_biased_users'] == max_users].iloc[0]
    
    print(f"   👥 Max biased users tested: {max_users:,}")
    print(f"   📈 RMSE change: {max_effect['rmse_change_pct']:+.2f}%")
    print(f"   📈 MAE change: {max_effect['mae_change_pct']:+.2f}%")
    
    if genre == 'adventure':
        print(f"   🗺️  Adventure rec change: {max_effect['adv_rec_change']:+.1f} percentage points")
        print(f"   🔍 Mystery rec change: {max_effect['mys_rec_change']:+.1f} percentage points")
    else:
        print(f"   🔍 Mystery rec change: {max_effect['mys_rec_change']:+.1f} percentage points")
        print(f"   🗺️  Adventure rec change: {max_effect['adv_rec_change']:+.1f} percentage points")
    
    # Calculate bias amplification ratio
    if genre == 'adventure':
        target_change = max_effect['adv_rec_change']
    else:
        target_change = max_effect['mys_rec_change']
    
    total_ratings = max_effect['total_ratings']
    synthetic_ratings = max_effect['synthetic_ratings']
    bias_ratio = (synthetic_ratings / total_ratings) * 100
    amplification = target_change / bias_ratio if bias_ratio > 0 else 0
    
    print(f"   📊 Synthetic ratings: {bias_ratio:.2f}% of total")
    print(f"   🔄 Amplification ratio: {amplification:.1f}x")

## 4. Threshold Analysis

In [ ]:
# Find minimum number of biased users for significant effect
def find_threshold(data, threshold_change=1.0):
    """
    Find minimum number of biased users for significant recommendation change
    """
    for _, row in data.sort_values('num_biased_users').iterrows():
        if row['genre'] == 'adventure':
            change = abs(row['adv_rec_change'])
        else:
            change = abs(row['mys_rec_change'])
        
        if change >= threshold_change:
            return row['num_biased_users'], change
    
    return None, 0

print("🎯 THRESHOLD ANALYSIS (Minimum users for 1% recommendation change)")
print("=" * 65)

for genre in ['adventure', 'mystery']:
    genre_data = biased_results[biased_results['genre'] == genre]
    threshold_users, change_at_threshold = find_threshold(genre_data, 1.0)
    
    if threshold_users:
        print(f"\n🎭 {genre.upper()} bias threshold: {threshold_users:,} users ({change_at_threshold:.1f}% change)")
    else:
        print(f"\n🎭 {genre.upper()} bias: No significant threshold found")

# Visualize threshold effects
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

for genre in ['adventure', 'mystery']:
    genre_data = biased_results[biased_results['genre'] == genre].sort_values('num_biased_users')
    
    if genre == 'adventure':
        y_values = genre_data['adv_rec_change']
        label = f'{genre.title()} → Adventure recs'
    else:
        y_values = genre_data['mys_rec_change']
        label = f'{genre.title()} → Mystery recs'
    
    ax.plot(genre_data['num_biased_users'], y_values, 
            marker='o', label=label, linewidth=2, markersize=6)

ax.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='1% threshold')
ax.axhline(y=-1.0, color='red', linestyle='--', alpha=0.5)
ax.set_xlabel('Number of Biased Users')
ax.set_ylabel('Recommendation Change (percentage points)')
ax.set_title('Bias Propagation Thresholds')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Cross-Genre Effects

In [ ]:
# Analyze cross-genre contamination effects
print("🔄 CROSS-GENRE CONTAMINATION ANALYSIS")
print("=" * 45)

# Adventure bias effects on Mystery recommendations
adv_bias_data = biased_results[biased_results['genre'] == 'adventure']
max_adv_bias = adv_bias_data[adv_bias_data['num_biased_users'] == adv_bias_data['num_biased_users'].max()].iloc[0]

print(f"\n🗺️  ADVENTURE BIAS ({max_adv_bias['num_biased_users']:,} users):")
print(f"   Adventure recs: {max_adv_bias['adv_rec_change']:+.1f} pp (primary effect)")
print(f"   Mystery recs: {max_adv_bias['mys_rec_change']:+.1f} pp (cross-contamination)")
print(f"   Other recs: {-(max_adv_bias['adv_rec_change'] + max_adv_bias['mys_rec_change']):+.1f} pp (displacement)")

# Mystery bias effects on Adventure recommendations
mys_bias_data = biased_results[biased_results['genre'] == 'mystery']
max_mys_bias = mys_bias_data[mys_bias_data['num_biased_users'] == mys_bias_data['num_biased_users'].max()].iloc[0]

print(f"\n🔍 MYSTERY BIAS ({max_mys_bias['num_biased_users']:,} users):")
print(f"   Mystery recs: {max_mys_bias['mys_rec_change']:+.1f} pp (primary effect)")
print(f"   Adventure recs: {max_mys_bias['adv_rec_change']:+.1f} pp (cross-contamination)")
print(f"   Other recs: {-(max_mys_bias['adv_rec_change'] + max_mys_bias['mys_rec_change']):+.1f} pp (displacement)")

# Create contamination matrix visualization
fig, ax = plt.subplots(1, 1, figsize=(10, 6))

# Prepare data for stacked bar chart
categories = ['Adventure Bias\n(2000 users)', 'Mystery Bias\n(2000 users)']
adventure_changes = [max_adv_bias['adv_rec_change'], max_mys_bias['adv_rec_change']]
mystery_changes = [max_adv_bias['mys_rec_change'], max_mys_bias['mys_rec_change']]
other_changes = [-(max_adv_bias['adv_rec_change'] + max_adv_bias['mys_rec_change']),
                -(max_mys_bias['adv_rec_change'] + max_mys_bias['mys_rec_change'])]

width = 0.6
x = np.arange(len(categories))

p1 = ax.bar(x, adventure_changes, width, label='Adventure recs', color='skyblue')
p2 = ax.bar(x, mystery_changes, width, bottom=adventure_changes, label='Mystery recs', color='lightcoral')
p3 = ax.bar(x, other_changes, width, bottom=np.array(adventure_changes) + np.array(mystery_changes), 
           label='Other recs', color='lightgray')

ax.set_ylabel('Recommendation Change (percentage points)')
ax.set_title('Cross-Genre Contamination Effects')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Summary and Conclusions

In [ ]:
# Generate comprehensive summary
print("📋 EXPERIMENTAL SUMMARY & CONCLUSIONS")
print("=" * 50)

# Overall system robustness
max_rmse_increase = biased_results['rmse_change_pct'].max()
max_mae_increase = biased_results['mae_change_pct'].max()

print(f"\n🎯 SYSTEM ROBUSTNESS:")
print(f"   Maximum RMSE increase: {max_rmse_increase:.2f}%")
print(f"   Maximum MAE increase: {max_mae_increase:.2f}%")
print(f"   Assessment: {'ROBUST' if max_rmse_increase < 5 else 'VULNERABLE'} to bias injection")

# Bias propagation effectiveness
max_adv_effect = biased_results[biased_results['genre'] == 'adventure']['adv_rec_change'].max()
max_mys_effect = biased_results[biased_results['genre'] == 'mystery']['mys_rec_change'].max()

print(f"\n🔄 BIAS PROPAGATION:")
print(f"   Adventure bias max effect: {max_adv_effect:.1f} pp")
print(f"   Mystery bias max effect: {max_mys_effect:.1f} pp")
print(f"   Propagation assessment: {'HIGH' if max(max_adv_effect, max_mys_effect) > 5 else 'MODERATE'}")

# Minimum effective bias size
adv_threshold, _ = find_threshold(biased_results[biased_results['genre'] == 'adventure'], 1.0)
mys_threshold, _ = find_threshold(biased_results[biased_results['genre'] == 'mystery'], 1.0)

print(f"\n⚖️ MINIMUM EFFECTIVE BIAS:")
print(f"   Adventure: {adv_threshold:,} users (1% effect)")
print(f"   Mystery: {mys_threshold:,} users (1% effect)")

# Attack feasibility assessment
total_users = 53424  # From original dataset
min_attack_size = min(adv_threshold, mys_threshold) if adv_threshold and mys_threshold else None

if min_attack_size:
    attack_percentage = (min_attack_size / total_users) * 100
    print(f"\n⚠️  ATTACK FEASIBILITY:")
    print(f"   Minimum attack size: {min_attack_size:,} users ({attack_percentage:.2f}% of total)")
    print(f"   Feasibility: {'HIGH' if attack_percentage < 1 else 'MODERATE' if attack_percentage < 5 else 'LOW'}")

print(f"\n📊 KEY FINDINGS:")
print(f"   1. SVD shows {'good' if max_rmse_increase < 2 else 'moderate'} robustness to bias injection")
print(f"   2. Bias propagation is {'effective' if max(max_adv_effect, max_mys_effect) > 3 else 'limited'} but measurable")
print(f"   3. Cross-genre contamination {'occurs' if abs(max_adv_bias['mys_rec_change']) > 0.5 else 'minimal'}")
print(f"   4. Small numbers of biased users can {'significantly' if min_attack_size and min_attack_size < 1000 else 'moderately'} impact recommendations")

# Save final summary
summary_stats = {
    'max_rmse_increase_pct': max_rmse_increase,
    'max_mae_increase_pct': max_mae_increase,
    'max_adventure_bias_effect': max_adv_effect,
    'max_mystery_bias_effect': max_mys_effect,
    'adventure_threshold_users': adv_threshold,
    'mystery_threshold_users': mys_threshold,
    'min_attack_size': min_attack_size,
    'attack_feasibility_pct': attack_percentage if min_attack_size else None
}

pd.DataFrame([summary_stats]).to_csv('../results/biased/experiment_summary.csv', index=False)

print(f"\n✅ Analysis complete! Summary saved to ../results/biased/experiment_summary.csv")